CACHEUX Nolan

In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

tsp_data = np.loadtxt('../TSPDataset/gr17.2085.tsp')  # Adjust path as needed
print("Data shape:", tsp_data.shape)


Data shape: (17, 17)


In [2]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt


n_cities = tsp_data.shape[0]
n_ants = 20
alpha = 1.0              # Importance of pheromone
beta = 2.0               # Importance of heuristic (1/distance)
evaporation_rate = 0.5   # Pheromone evaporation rate
Q = 1.0                  # Pheromone deposit factor
n_iterations = 100
random_seed = 42         # For reproducibility

random.seed(random_seed)
np.random.seed(random_seed)

# INITIALIZE PHEROMONE & HEURISTIC MATRICES
pheromone = np.ones((n_cities, n_cities))
heuristic = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        if i != j:
            # If the distance is zero, add a small number to avoid division by zero.
            heuristic[i][j] = 1.0 / (tsp_data[i][j] + 1e-10)
        else:
            heuristic[i][j] = 0.0

best_route = None
best_distance = float('inf')


def select_next_city(current_city, unvisited):
    """
    Select the next city based on roulette wheel selection using
    (pheromone^alpha) * (heuristic^beta) probabilities.
    """
    pheromone_vals = np.array([pheromone[current_city][u] for u in unvisited])
    heuristic_vals = np.array([heuristic[current_city][u] for u in unvisited])
    probs = (pheromone_vals ** alpha) * (heuristic_vals ** beta)
    probs_sum = np.sum(probs)
    if probs_sum == 0:
        return random.choice(list(unvisited))
    probs = probs / probs_sum

    r = random.random()
    cumulative = 0.0
    for i, city in enumerate(unvisited):
        cumulative += probs[i]
        if r <= cumulative:
            return city
    return list(unvisited)[-1]


def update_pheromone(routes):
    global pheromone
    # Evaporate pheromone on all edges
    pheromone *= (1 - evaporation_rate)
    # Deposit new pheromone for each ant's route
    for route in routes:
        # Compute the route's total distance inline
        d = 0.0
        for i in range(len(route) - 1):
            d += tsp_data[route[i]][route[i+1]]
        d += tsp_data[route[-1]][route[0]]  # Return to start

        deposit = Q / d
        # Deposit pheromone for each edge in the route (including the return edge)
        for i in range(len(route) - 1):
            a, b = route[i], route[i+1]
            pheromone[a][b] += deposit
            pheromone[b][a] += deposit
        pheromone[route[-1]][route[0]] += deposit
        pheromone[route[0]][route[-1]] += deposit


for iteration in range(n_iterations):
    all_routes = []
    for _ in range(n_ants):
        # ---- Construct a route (inline, without a separate function) ----
        route = []
        start_city = random.randint(0, n_cities - 1)
        route.append(start_city)
        unvisited = set(range(n_cities))
        unvisited.remove(start_city)
        current_city = start_city

        # Build the route until all cities are visited
        while unvisited:
            next_city = select_next_city(current_city, unvisited)
            route.append(next_city)
            unvisited.remove(next_city)
            current_city = next_city

        all_routes.append(route)

        # ---- Compute the route's total distance inline (without a separate function) ----
        d = 0.0
        for i in range(len(route) - 1):
            d += tsp_data[route[i]][route[i+1]]
        d += tsp_data[route[-1]][route[0]]  # complete the tour

        if d < best_distance:
            best_distance = d
            best_route = route

    update_pheromone(all_routes)
    # Uncomment to see progress: 
    # print(f"Iteration {iteration+1}/{n_iterations}, Best distance: {best_distance:.2f}")

print("Best route:", best_route)
print("Best distance:", best_distance)



Best route: [12, 6, 7, 5, 16, 13, 14, 2, 10, 9, 1, 4, 8, 11, 15, 0, 3]
Best distance: 2085.0


In [3]:
%pip install python-tsp

In [4]:
# Compare with a python-tsp library solution
from python_tsp.exact import solve_tsp_dynamic_programming

permutation, distance_lib = solve_tsp_dynamic_programming(tsp_data)
print("python-tsp Results (Exact DP)")
print("Permutation (python-tsp):", permutation)
print("Distance (python-tsp):", distance_lib)


python-tsp Results (Exact DP)
Permutation (python-tsp): [0, 3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 9, 1, 4, 8, 11, 15]
Distance (python-tsp): 2085.0
